In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
def subsets(s):
    sets = []
    for i in range(1 << len(s)):
        subset = [s[bit] for bit in range(len(s)) if is_bit_set(i, bit)]
        sets.append(subset)
    return sets

def halfsubsets(s):
    sets = []
    for i in range(1 << len(s)):
        subset = [s[bit] for bit in range(len(s)) if is_bit_set(i, bit)]
        if(len(subset) <= len(s)/2):
            sets.append(subset)
    return sets

def is_bit_set(num, bit):
    return num & (1 << bit) > 0

# I. Simple Test Data

In [3]:
df = pd.read_csv('SimpleTestData.csv')
df

,Y,A,X,P(A|X),D_A,D_B,D_A = A,D_B = B
0,1,1,1,0.5,2,2,0,0
1,2,1,2,0.5,1,2,1,0
2,3,1,3,0.5,1,1,1,1
3,4,1,4,0.5,1,1,1,1
4,5,1,5,0.5,1,1,1,1
5,3,2,1,0.5,2,2,1,1
6,3,2,2,0.5,1,2,0,1
7,3,2,3,0.5,1,1,0,0
8,3,2,4,0.5,1,1,0,0
9,3,2,5,0.5,1,1,0,0


### Part 1. T0 validation

In [4]:
sumY_A1 = df[df['A'] == 1]['Y'].sum()
sumY_A2 = df[df['A'] == 2]['Y'].sum()
sumY_A = df['Y'].sum()
print("Sum of Y given Action 1 : " + str(sumY_A1))
print("Sum of Y given Action 2 : " + str(sumY_A2))
print("Sum of Y : " + str(sumY_A))

Sum of Y given Action 1 : 15
Sum of Y given Action 2 : 15
Sum of Y : 30


### Part 2. Cut info validation

In [5]:
unique_X = df['X'].unique()
unique_X

array([1, 2, 3, 4, 5], dtype=int64)

In [6]:
print("Cut Range for X: "+str(unique_X+1))
print("Cut size: "+str(len(unique_X)))

Cut Range for X: [2 3 4 5 6]
Cut size: 5


### Part 3. Lookup table validation

In [7]:
for i in unique_X+1:
    df['Cut'+str(i)] = df['X'].apply(lambda x : 1 if x<i else 0)

In [8]:
df

,Y,A,X,P(A|X),D_A,D_B,D_A = A,D_B = B,Cut2,Cut3,Cut4,Cut5,Cut6
0,1,1,1,0.5,2,2,0,0,1,1,1,1,1
1,2,1,2,0.5,1,2,1,0,0,1,1,1,1
2,3,1,3,0.5,1,1,1,1,0,0,1,1,1
3,4,1,4,0.5,1,1,1,1,0,0,0,1,1
4,5,1,5,0.5,1,1,1,1,0,0,0,0,1
5,3,2,1,0.5,2,2,1,1,1,1,1,1,1
6,3,2,2,0.5,1,2,0,1,0,1,1,1,1
7,3,2,3,0.5,1,1,0,0,0,0,1,1,1
8,3,2,4,0.5,1,1,0,0,0,0,0,1,1
9,3,2,5,0.5,1,1,0,0,0,0,0,0,1


# Load CSV Data

In [9]:
df = pd.read_csv('TestSample_100_XTable.csv')
del df['Unnamed: 0']
df

,ID,Cont1,Cont2,Cont3,Ord1,Ord2,Ord3,Nom1,Nom2,Nom3,A,Y
0,1,0,1,7,4,2,0,4,3,3,0,31.21120
1,2,9,0,3,0,3,3,2,3,3,0,20.84580
2,3,2,7,0,1,3,3,4,1,3,0,94.73210
3,4,2,3,0,0,0,2,3,2,0,1,22.41790
4,5,4,4,2,2,3,4,1,4,1,1,15.07230
5,6,9,4,1,4,0,2,2,3,4,1,21.51730
6,7,1,2,9,3,1,2,2,0,4,0,29.96790
7,8,1,8,2,3,1,0,4,3,3,0,0.48750
8,9,0,2,8,0,2,0,1,4,1,1,63.68780
9,10,8,4,3,4,1,4,1,4,2,1,19.11400


### Part 1. T0 validation

In [10]:
A0_N = sum(x==0 for x in df['A'])
A1_N = sum(x==1 for x in df['A']) 
print(A0_N + A1_N == df.shape[0])
print("Number of Action 0 : " + str(A0_N))
print("Number of Action 1 : " + str(A1_N))

True
Number of Action 0 : 60
Number of Action 1 : 40


In [11]:
sumY_A0 = df[df['A'] == 0]['Y'].sum()
sumY_A1 = df[df['A'] == 1]['Y'].sum()
sumY_A = df['Y'].sum()
print(math.isclose(sumY_A0 + sumY_A1, sumY_A, rel_tol=1e-5))
print("Sum of Y given Action 0 : " + str(sumY_A0))
print("Sum of Y given Action 1 : " + str(sumY_A1))
print("Sum of Y : " + str(sumY_A))

True
Sum of Y given Action 0 : 2841.77937
Sum of Y given Action 1 : 1706.1182
Sum of Y : 4547.897570000002


### Part 2. Cut info validation

In [12]:
unique_X0 = sorted((df['Cont1']+1).unique())
print("Cut Range for X0: "+str(unique_X0))
print("Cut size for X0: "+str(len(unique_X0)))

Cut Range for X0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Cut size for X0: 10


In [13]:
unique_X1 = sorted((df['Cont2']+1).unique())
print("Cut Range for X1: "+str(unique_X1))
print("Cut size for X1: "+str(len(unique_X1)))

Cut Range for X1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Cut size for X1: 10


In [14]:
unique_X2 = sorted((df['Cont3']+1).unique())
print("Cut Range for X2: "+str(unique_X2))
print("Cut size for X2: "+str(len(unique_X2)))

Cut Range for X2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Cut size for X2: 10


In [15]:
unique_X3 = sorted((df['Ord1']+1).unique())
print("Cut Range for X3: "+str(unique_X3))
print("Cut size for X3: "+str(len(unique_X3)))

Cut Range for X3: [1, 2, 3, 4, 5]
Cut size for X3: 5


In [16]:
unique_X4 = sorted((df['Ord2']+1).unique())
print("Cut Range for X4: "+str(unique_X4))
print("Cut size for X4: "+str(len(unique_X4)))

Cut Range for X4: [1, 2, 3, 4, 5]
Cut size for X4: 5


In [17]:
unique_X5 = sorted((df['Ord3']+1).unique())
print("Cut Range for X5: "+str(unique_X5))
print("Cut size for X5: "+str(len(unique_X5)))

Cut Range for X5: [1, 2, 3, 4, 5]
Cut size for X5: 5


In [18]:
unique_X6 = sorted(df['Nom1'].unique())

In [19]:
sets_X6 = subsets(unique_X6)
sets_X6

[[],
 [0],
 [1],
 [0, 1],
 [2],
 [0, 2],
 [1, 2],
 [0, 1, 2],
 [3],
 [0, 3],
 [1, 3],
 [0, 1, 3],
 [2, 3],
 [0, 2, 3],
 [1, 2, 3],
 [0, 1, 2, 3],
 [4],
 [0, 4],
 [1, 4],
 [0, 1, 4],
 [2, 4],
 [0, 2, 4],
 [1, 2, 4],
 [0, 1, 2, 4],
 [3, 4],
 [0, 3, 4],
 [1, 3, 4],
 [0, 1, 3, 4],
 [2, 3, 4],
 [0, 2, 3, 4],
 [1, 2, 3, 4],
 [0, 1, 2, 3, 4]]

In [20]:
sets_X6 = halfsubsets(unique_X6)
print("Cut Range for X6: "+str(list(range(len(sets_X6)))))
print("Cut Size for X6: "+str(len(sets_X6)))
sets_X6

Cut Range for X6: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cut Size for X6: 16


[[],
 [0],
 [1],
 [0, 1],
 [2],
 [0, 2],
 [1, 2],
 [3],
 [0, 3],
 [1, 3],
 [2, 3],
 [4],
 [0, 4],
 [1, 4],
 [2, 4],
 [3, 4]]

In [21]:
unique_X7 = sorted(df['Nom2'].unique())
sets_X7 = halfsubsets(unique_X7)
print("Cut Range for X7: "+str(list(range(len(sets_X7)))))
print("Cut Size for X7: "+str(len(sets_X7)))
sets_X7

Cut Range for X7: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cut Size for X7: 16


[[],
 [0],
 [1],
 [0, 1],
 [2],
 [0, 2],
 [1, 2],
 [3],
 [0, 3],
 [1, 3],
 [2, 3],
 [4],
 [0, 4],
 [1, 4],
 [2, 4],
 [3, 4]]

In [22]:
unique_X8 = sorted(df['Nom3'].unique())
sets_X8 = halfsubsets(unique_X8)
print("Cut Range for X8: "+str(list(range(len(sets_X8)))))
print("Cut Size for X8: "+str(len(sets_X8)))
sets_X8

Cut Range for X8: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cut Size for X8: 16


[[],
 [0],
 [1],
 [0, 1],
 [2],
 [0, 2],
 [1, 2],
 [3],
 [0, 3],
 [1, 3],
 [2, 3],
 [4],
 [0, 4],
 [1, 4],
 [2, 4],
 [3, 4]]

### Part 3. Lookup table validation

In [23]:
headers = list(df)[1:10]
headers

['Cont1', 'Cont2', 'Cont3', 'Ord1', 'Ord2', 'Ord3', 'Nom1', 'Nom2', 'Nom3']

In [24]:
X = df[headers]
X

,Cont1,Cont2,Cont3,Ord1,Ord2,Ord3,Nom1,Nom2,Nom3
0,0,1,7,4,2,0,4,3,3
1,9,0,3,0,3,3,2,3,3
2,2,7,0,1,3,3,4,1,3
3,2,3,0,0,0,2,3,2,0
4,4,4,2,2,3,4,1,4,1
5,9,4,1,4,0,2,2,3,4
6,1,2,9,3,1,2,2,0,4
7,1,8,2,3,1,0,4,3,3
8,0,2,8,0,2,0,1,4,1
9,8,4,3,4,1,4,1,4,2


#### 3.1 Cuts for continuous  variable 1

In [25]:
Cut_X0 = X.copy()[['Cont1']];
for i in unique_X0:
    Cut_X0['Cut'+str(i)] = Cut_X0['Cont1'].apply(lambda x : 1 if x<i else 0)

In [26]:
Cut_X0.head(n=20)

,Cont1,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10
0,0,1,1,1,1,1,1,1,1,1,1
1,9,0,0,0,0,0,0,0,0,0,1
2,2,0,0,1,1,1,1,1,1,1,1
3,2,0,0,1,1,1,1,1,1,1,1
4,4,0,0,0,0,1,1,1,1,1,1
5,9,0,0,0,0,0,0,0,0,0,1
6,1,0,1,1,1,1,1,1,1,1,1
7,1,0,1,1,1,1,1,1,1,1,1
8,0,1,1,1,1,1,1,1,1,1,1
9,8,0,0,0,0,0,0,0,0,1,1


In [27]:
print("Cont1 Cut1:")
Cut_X0['Cut1'].values

Cont1 Cut1:


array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [28]:
print("Cont1 Cut4:")
Cut_X0['Cut4'].values

Cont1 Cut4:


array([1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0], dtype=int64)

In [29]:
print("Cont1 Cut10:")
Cut_X0['Cut10'].values

Cont1 Cut10:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.2 Cuts for continuous  variable 2

In [30]:
Cut_X1 = X.copy()[['Cont2']];
for i in unique_X1:
    Cut_X1['Cut'+str(i)] = Cut_X1['Cont2'].apply(lambda x : 1 if x<i else 0)
Cut_X1.head(n=20)

,Cont2,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10
0,1,0,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1,1,1
2,7,0,0,0,0,0,0,0,1,1,1
3,3,0,0,0,1,1,1,1,1,1,1
4,4,0,0,0,0,1,1,1,1,1,1
5,4,0,0,0,0,1,1,1,1,1,1
6,2,0,0,1,1,1,1,1,1,1,1
7,8,0,0,0,0,0,0,0,0,1,1
8,2,0,0,1,1,1,1,1,1,1,1
9,4,0,0,0,0,1,1,1,1,1,1


In [31]:
print("Cont2 Cut1:")
Cut_X1['Cut1'].values

Cont2 Cut1:


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [32]:
print("Cont2 Cut5:")
Cut_X1['Cut5'].values

Cont2 Cut5:


array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [33]:
print("Cont2 Cut10:")
Cut_X1['Cut10'].values

Cont2 Cut10:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.3 Cuts for continuous  variable 3

In [34]:
Cut_X2 = X.copy()[['Cont3']];
for i in unique_X2:
    Cut_X2['Cut'+str(i)] = Cut_X2['Cont3'].apply(lambda x : 1 if x<i else 0)
Cut_X2.head(n=20)

,Cont3,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10
0,7,0,0,0,0,0,0,0,1,1,1
1,3,0,0,0,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,1
3,0,1,1,1,1,1,1,1,1,1,1
4,2,0,0,1,1,1,1,1,1,1,1
5,1,0,1,1,1,1,1,1,1,1,1
6,9,0,0,0,0,0,0,0,0,0,1
7,2,0,0,1,1,1,1,1,1,1,1
8,8,0,0,0,0,0,0,0,0,1,1
9,3,0,0,0,1,1,1,1,1,1,1


In [35]:
print("Cont3 Cut1:")
Cut_X2['Cut1'].values

Cont3 Cut1:


array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [36]:
print("Cont3 Cut6:")
Cut_X2['Cut6'].values

Cont3 Cut6:


array([0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0], dtype=int64)

In [37]:
print("Cont3 Cut10:")
Cut_X2['Cut10'].values

Cont3 Cut10:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.4 Cuts for ordinal variable 1

In [38]:
Cut_X3 = X.copy()[['Ord1']];
for i in unique_X3:
    Cut_X3['Cut'+str(i)] = Cut_X3['Ord1'].apply(lambda x : 1 if x<i else 0)
Cut_X3.head(n=20)

,Ord1,Cut1,Cut2,Cut3,Cut4,Cut5
0,4,0,0,0,0,1
1,0,1,1,1,1,1
2,1,0,1,1,1,1
3,0,1,1,1,1,1
4,2,0,0,1,1,1
5,4,0,0,0,0,1
6,3,0,0,0,1,1
7,3,0,0,0,1,1
8,0,1,1,1,1,1
9,4,0,0,0,0,1


In [39]:
print("Ord1 Cut1:")
Cut_X3['Cut1'].values

Ord1 Cut1:


array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)

In [40]:
print("Ord1 Cut2:")
Cut_X3['Cut2'].values

Ord1 Cut2:


array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)

In [41]:
print("Ord1 Cut5:")
Cut_X3['Cut5'].values

Ord1 Cut5:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.5 Cuts for ordinal variable 2

In [42]:
Cut_X4 = X.copy()[['Ord2']];
for i in unique_X4:
    Cut_X4['Cut'+str(i)] = Cut_X4['Ord2'].apply(lambda x : 1 if x<i else 0)
Cut_X4.head(n=20)

,Ord2,Cut1,Cut2,Cut3,Cut4,Cut5
0,2,0,0,1,1,1
1,3,0,0,0,1,1
2,3,0,0,0,1,1
3,0,1,1,1,1,1
4,3,0,0,0,1,1
5,0,1,1,1,1,1
6,1,0,1,1,1,1
7,1,0,1,1,1,1
8,2,0,0,1,1,1
9,1,0,1,1,1,1


In [43]:
print("Ord2 Cut1:")
Cut_X4['Cut1'].values

Ord2 Cut1:


array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1], dtype=int64)

In [44]:
print("Ord2 Cut3:")
Cut_X4['Cut3'].values

Ord2 Cut3:


array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1], dtype=int64)

In [45]:
print("Ord2 Cut5:")
Cut_X4['Cut5'].values

Ord2 Cut5:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.6 Cuts for ordinal variable 3

In [46]:
Cut_X5 = X.copy()[['Ord3']];
for i in unique_X5:
    Cut_X5['Cut'+str(i)] = Cut_X5['Ord3'].apply(lambda x : 1 if x<i else 0)
Cut_X5.head(n=20)

,Ord3,Cut1,Cut2,Cut3,Cut4,Cut5
0,0,1,1,1,1,1
1,3,0,0,0,1,1
2,3,0,0,0,1,1
3,2,0,0,1,1,1
4,4,0,0,0,0,1
5,2,0,0,1,1,1
6,2,0,0,1,1,1
7,0,1,1,1,1,1
8,0,1,1,1,1,1
9,4,0,0,0,0,1


In [47]:
print("Ord3 Cut1:")
Cut_X5['Cut1'].values

Ord3 Cut1:


array([1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [48]:
print("Ord3 Cut4:")
Cut_X5['Cut4'].values

Ord3 Cut4:


array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [49]:
print("Ord3 Cut5:")
Cut_X5['Cut5'].values

Ord3 Cut5:


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

#### 3.7 Cuts for nominal variable 1

In [50]:
Cut_X6 = X.copy()[['Nom1']];
for i in range(len(sets_X6)):
    Cut_X6['Cut'+str(i)] = Cut_X6['Nom1'].apply(lambda x : 1 if x in sets_X6[i] else 0)
Cut_X6.head(n=20)

,Nom1,Cut0,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10,Cut11,Cut12,Cut13,Cut14,Cut15
0,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
1,2,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0
2,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
3,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
4,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
5,2,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0
6,2,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0
7,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
8,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
9,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0


In [51]:
print("Nom1 Cut0:")
Cut_X6['Cut0'].values

Nom1 Cut0:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [52]:
print("Nom1 Cut3:")
Cut_X6['Cut3'].values

Nom1 Cut3:


array([0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0], dtype=int64)

In [53]:
print("Nom1 Cut15:")
Cut_X6['Cut15'].values

Nom1 Cut15:


array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1], dtype=int64)

#### 3.8 Cuts for nominal variable 2

In [54]:
Cut_X7 = X.copy()[['Nom2']];
for i in range(len(sets_X7)):
    Cut_X7['Cut'+str(i)] = Cut_X7['Nom2'].apply(lambda x : 1 if x in sets_X7[i] else 0)
Cut_X7.head(n=20)

,Nom2,Cut0,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10,Cut11,Cut12,Cut13,Cut14,Cut15
0,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
1,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
2,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
3,2,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
5,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
6,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0
7,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
8,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
9,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1


In [55]:
print("Nom2 Cut0:")
Cut_X7['Cut0'].values

Nom2 Cut0:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [56]:
print("Nom2 Cut6:")
Cut_X7['Cut6'].values

Nom2 Cut6:


array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [57]:
print("Nom2 Cut15:")
Cut_X7['Cut15'].values

Nom2 Cut15:


array([1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

#### 3.9 Cuts for nominal variable 3

In [58]:
Cut_X8 = X.copy()[['Nom3']];
for i in range(len(sets_X8)):
    Cut_X8['Cut'+str(i)] = Cut_X8['Nom3'].apply(lambda x : 1 if x in sets_X8[i] else 0)
Cut_X8.head(n=20)

,Nom3,Cut0,Cut1,Cut2,Cut3,Cut4,Cut5,Cut6,Cut7,Cut8,Cut9,Cut10,Cut11,Cut12,Cut13,Cut14,Cut15
0,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
1,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
2,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
3,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0
4,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
5,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
6,4,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
7,3,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1
8,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
9,2,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0


In [59]:
print("Nom3 Cut0:")
Cut_X8['Cut0'].values

Nom3 Cut0:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [60]:
print("Nom3 Cut9:")
Cut_X8['Cut9'].values

Nom3 Cut9:


array([1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [61]:
print("Nom3 Cut15:")
Cut_X8['Cut15'].values

Nom3 Cut15:


array([1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0], dtype=int64)